In [1]:
!pip install fastapi nest-asyncio pyngrok uvicorn

In [2]:
import numpy as np
from fastapi import FastAPI, Form
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import pandas as pd
from starlette.responses import HTMLResponse
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import re

In [3]:
app = FastAPI()

@app.get('/predict', response_class=HTMLResponse)
def take_inp():
    return '''
        <form method="post">
        <input maxlength="28" name="text" type="text" value="Text Emotion to be tested" />
        <input type="submit" />'''

In [4]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: http://22821470d305.ngrok.io


INFO:     Started server process [326]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [326]


In [6]:
data = pd.read_csv('Sentiment.csv')
tokenizer = Tokenizer(num_words=2000, split=' ')
tokenizer.fit_on_texts(data['text'].values)
#We will create a function to remove unwanted characters in Tweets using Regex.
#We will use Tensorflow’s tokenizer to tokenize our dataset, and Tensorflow’s pad_sequences to pad our sequences.

def preProcess_data(text):
    text = text.lower()
    new_text = re.sub('[^a-zA-z0-9\s]','',text)
    new_text = re.sub('rt', '', new_text)
    return new_text

def my_pipeline(text):
    text_new = preProcess_data(text)
    X = tokenizer.texts_to_sequences(pd.Series(text_new).values)
    X = pad_sequences(X, maxlen=28)
    return X

In [7]:
!pip install python-multipart

In [8]:
@app.post('/predict')
def predict(text:str = Form(...)):
    clean_text = my_pipeline(text) #clean, and preprocess the text through pipeline
    loaded_model = tf.keras.models.load_model('sentiment.h5') #load the saved model 
    predictions = loaded_model.predict(clean_text) #predict the text
    sentiment = int(np.argmax(predictions)) #calculate the index of max sentiment
    probability = max(predictions.tolist()[0]) #calulate the probability
    if sentiment==0:
         t_sentiment = 'negative' #set appropriate sentiment
    elif sentiment==1:
         t_sentiment = 'neutral'
    elif sentiment==2:
         t_sentiment='postive'
    return { #return the dictionary for endpoint
         "ACTUALL SENTENCE": text,
         "PREDICTED SENTIMENT": t_sentiment,
         "Probability": probability
    }
@app.get('/')
def basic_view():
    return {"WELCOME": "GO TO /docs route, or /post or send post request to /predict "}

In [9]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: http://f97cbb5b0421.ngrok.io


INFO:     Started server process [326]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     102.110.22.102:0 - "GET / HTTP/1.1" 200 OK
INFO:     102.110.22.102:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     102.110.22.102:0 - "GET /predict HTTP/1.1" 200 OK
INFO:     102.110.22.102:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     102.110.22.102:0 - "GET /predict HTTP/1.1" 200 OK
INFO:     102.110.22.102:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     102.110.22.102:0 - "GET /predict HTTP/1.1" 200 OK
INFO:     102.110.22.102:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     102.110.22.102:0 - "GET /predict HTTP/1.1" 200 OK
INFO:     102.110.22.102:0 - "POST /predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [326]
